In [3]:
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126
%pip install torch-geometric

Base.Meta.ParseError: ParseError:
# Error @ c:\Users\lucdo\graph_based_recsys\notebooks\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:1:1
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126
╙ ── not a unary operator

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch_geometric.utils import negative_sampling

class MLPBaseline(nn.Module):
    def __init__(self, in_features):
        super(MLPBaseline, self).__init__()
        input_dim = in_features * 2
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 256), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, 128), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x, edge_index):
        src_features = x[edge_index[0]]
        dst_features = x[edge_index[1]]
        combined_features = torch.cat([src_features, dst_features], dim=1)
        return self.layers(combined_features)

def prepare_mlp_data(train_pos_edge_index, num_nodes, batch_size):
    print("Generating negative samples...")
    train_neg_edge_index = negative_sampling(
        edge_index=train_pos_edge_index, num_nodes=num_nodes,
        num_neg_samples=train_pos_edge_index.shape[1], method='sparse'
    )
    train_edges = torch.cat([train_pos_edge_index, train_neg_edge_index], dim=1)
    train_labels = torch.cat([
        torch.ones(train_pos_edge_index.shape[1]),
        torch.zeros(train_neg_edge_index.shape[1])
    ])
    print(f"Total training samples: {train_labels.shape[0]}")
    dataset = TensorDataset(train_edges.t(), train_labels)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader

# --- 1. Load Day 1 Data ---
# (This is simulated; we need to load actual data here)
NUM_NODES = 548552
NUM_FEATURES = 398
NUM_TRAIN_EDGES = 1252107
BATCH_SIZE = 1024
LEARNING_RATE = 1e-3
EPOCHS = 3 # Start with a few to test

print("Loading dummy data (replace with actual data)")
node_features = torch.randn(NUM_NODES, NUM_FEATURES)
train_pos_edge_index = torch.randint(0, NUM_NODES, (2, NUM_TRAIN_EDGES)) 

# --- 2. Setup ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Prepare DataLoader
train_loader = prepare_mlp_data(train_pos_edge_index, NUM_NODES, BATCH_SIZE)

# Initialize Model, Loss, Optimizer
model = MLPBaseline(in_features=NUM_FEATURES).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.BCEWithLogitsLoss() # Use this loss for stability

# **CRITICAL**: Move the entire feature matrix to the GPU once.
# This is memory-intensive but necessary for this method.
node_features = node_features.to(device)

print("Starting training...")

# --- 3. Training Loop ---
model.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for i, (batch_edges, batch_labels) in enumerate(train_loader):
        
        # Data is [B, 2], needs to be [2, B] for the model
        batch_edges = batch_edges.t().to(device)
        batch_labels = batch_labels.to(device)
        
        # --- Forward Pass ---
        # Pass *all* features and the *current batch* of edges
        logits = model(node_features, batch_edges)
        
        # --- Calculate Loss ---
        loss = loss_fn(logits.squeeze(), batch_labels.float())
        
        # --- Backward Pass ---
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        if (i + 1) % 100 == 0:
            print(f"  Epoch {epoch+1}/{EPOCHS}, Batch {i+1}/{len(train_loader)}, Loss: {loss.item():.4f}")
            
    avg_loss = total_loss / len(train_loader)
    print(f"--- Epoch {epoch+1} Complete --- Avg Loss: {avg_loss:.4f} ---")

print("Training finished.")

# To save your model:
# torch.save(model.state_dict(), 'mlp_baseline.pth')

ArgumentError: ArgumentError: Package torch not found in current path.
- Run `import Pkg; Pkg.add("torch")` to install the torch package.